In [1]:
#1
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

park = Customer('Park', 100)
park

Customer(name='Park', fidelity=100)

In [7]:
class LineItem:
  def __init__(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity

In [17]:
class Order:
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = cart
    self.promotion = promotion

  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion.discount(self)
    return self.total() - discount

  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())


In [4]:
class Promotion(ABC):
  """할인 혜택 클래스들의 형태를 선언"""
  @abstractmethod
  def discount(self, order):
    """할인액을 구체적인 숫자로 반환"""
    pass

In [10]:
class FidelityPromo(Promotion):
  def discount(self, order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
  def discount(self, order):
    discount = 0
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total() * 0.1
    return discount

class LargeOrderPromo(Promotion):
  def discount(self, order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
      return order.total() * 0.07
    return 0


In [8]:
#2
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [18]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [19]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [20]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [22]:
long_order = [LineItem(str(item_code), 1, 1.0)
for item_code in range(10)]

Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [24]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [28]:
#3
class Order:
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion(self)
    return self.total() - discount
  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

In [26]:
def fidelity_promo(order):
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
    return discount
def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

In [29]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [30]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [31]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [33]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [42]:
#4
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
  """최대로 할인받을 금액을 반환"""
  return max(promo(order) for promo in promos)

In [35]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [36]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [37]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [40]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', 'park', '_1', '_i2', 'LineItem', '_i3', 'Order', '_i4', 'Promotion', '_i5', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i6', 'joe', 'ann', '_i7', '_i8', 'cart', '_i9', '_9', '_i10', '_i11', '_11', '_i12', '_12', '_i13', '_13', '_i14', '_14', '_i15', '_15', '_i16', '_16', '_i17', '_i18', '_18', '_i19', '_19', '_i20', 'banana_cart', '_20', '_i21', 'long_order', '_i22', '_22', '_i23', '_i24', '_24', '_i25', '_i26', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i27', '_27', '_i28', '_i29', '_29', '_i30', '_30', '_i31', '_31', '_i32', '_i33', '_33', '_i34', 'promos', 'best_promo', '_i35', '_35', '_i36', '_36', '_i37', '_37', '_i38', '_i39', '_39', '_i40'])

In [43]:
globals() ['best_promo']

<function __main__.best_promo(order)>

In [44]:
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]